# GFK

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#adaptacion
from utils.adaptacion import *

#clasificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from utils.clasificacion import *

#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#otros
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib

#variables para guardar los resultados
tipo = pruebas[2]
dataset_name = datasets[0]
dims = dimensions[dataset_name]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Pruebas con el dataset Amazon

In [3]:
print tipo
print dataset_name
print dims
print data_path

gfk
amazon
5000
data


In [4]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
unlabeled = dataset_object.unlabeled
domains = dataset_object.domains

Dataset already splitted


### Creación de modelos de adaptación.

In [5]:
#por cada par se entrena un adaptador y se guarda el que mejor resultados entrega

size_dims = [int(dims/4), int(dims/2)]
n_subs = [10, 50, 100]

parameters = {
    'dims': size_dims,
    'n_subs': n_subs
}


print "Adaptando entre:"
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "%s - %s" % (src, tgt)
            X_tr = np.asarray(labeled[src]['X_tr'].todense())
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            gfk, score = gfk_grid_search(parameters, X_tr, y_tr, X_ts)
            print "Mejor modelo: %.3f" % score
            
            
            gfk_save_path = os.path.join(models_path, tipo, "%s_%s_%s.pkl" % (dataset_name, src,tgt))
            print "Guardando modelo en %s" % gfk_save_path
            joblib.dump(gfk, gfk_save_path)

print "\nAdaptaciones completadas"

Adaptando entre:
electronics - dvd
Mejor modelo: 0.753
Guardando modelo en models/gfk/amazon_electronics_dvd.pkl
electronics - kitchen


MemoryError: 

### Pruebas de clasificación

In [ ]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de 2" % (i+1)
            
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'][:, :dims].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'][:, :dims].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            model_name = "indomain_%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, model_name)
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            #############
            #### GFK ####
            #############
            # se adaptan los dominios usando GFK
            print "Adaptando dominios..."
            best_model_path = os.path.join(models_path, tipo, "%s_%s_%s.pkl" % (dataset_name, src, tgt))
            gfk = load_best_model(best_model_path)
            
            
            X_tr = np.asarray(labeled[src]['X_tr'][:, :dims].todense())
            X_tr_a = transform_gfk(X_tr, gfk)
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'][:, :dims].todense())
            X_ts_a = transform_gfk(X_ts, gfk)
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            clf = get_best_score(X_tr_a, y_tr, clasifier='KNeighbors')
            t_error = 1-clf.score(X_ts_a, y_ts)
            #t_error = gfk_compute_accuracy(gfk.G, X_tr, y_tr, X_ts, y_ts)
            
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['GFK',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            
            i += 1
            
print "Pruebas completadas."

In [15]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,GFK,k->d,kitchen,dvd,27.998200,51.50,23.501800
1,GFK,k->e,kitchen,electronics,28.688278,55.00,26.311722
2,GFK,k->b,kitchen,books,32.241517,57.25,25.008483
3,GFK,d->k,dvd,kitchen,23.989994,54.50,30.510006
4,GFK,d->e,dvd,electronics,28.688278,57.25,28.561722
5,GFK,d->b,dvd,books,32.241517,55.25,23.008483
6,GFK,e->k,electronics,kitchen,23.989994,54.75,30.760006
7,GFK,e->d,electronics,dvd,27.998200,53.25,25.251800
8,GFK,e->b,electronics,books,32.241517,54.50,22.258483
9,GFK,b->k,books,kitchen,23.989994,52.25,28.260006


In [17]:
new_scores_path = os.path.join(scores_path, tipo, dataset_name+".csv")

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/gfk/amazon.csv
Resultados guardados.
